In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.svm import SVC
import scipy
from dataset import load_dataset, load_labels, split_data, format_labels
from features import time_series_features, hjorth_features
from sklearn.metrics import confusion_matrix, accuracy_score
import variables as vi
import matplotlib.pyplot as plt


def calculate_stress_level():
    data_type = "wt_filtered"
    test_type = "Arithmetic"

    dataset_ = load_dataset(data_type=data_type, test_type=test_type)
    dataset = split_data(dataset_, vi.SFREQ)

    label_ = load_labels()
    label = format_labels(label_, test_type=test_type, epochs=dataset.shape[1])

    features = time_series_features(dataset)
    random_index = np.random.randint(0, 96)
    random_data = dataset[random_index]
    random_features = hjorth_features(np.expand_dims(random_data, axis=1))

    stressed_data = features[np.where(label != 0)]
    non_stressed_data = features[np.where(label == 0)]
    P = np.zeros(features.shape[1])
    for j, i in enumerate(range(features.shape[1])):
        _, p = scipy.stats.ttest_ind(stressed_data[:, i], non_stressed_data[:, i])
        P[j] = p

    p_value = P[random_index]
    if p_value < 0.01:
        stress_level = "VERY HIGH!!!!!"
    elif p_value < 0.05:
        stress_level = "HIGH!!!"
    elif p_value < 0.10:
        stress_level = "MEDIUM"
    elif p_value < 0.20:
        stress_level = "LOW"
    else:
        stress_level = "No Stress"
        
    s = "NEED DOCTOR SUPERVISION"
    p = "NO NEED DOCTOR SUPERVISION"

    messagebox.showinfo("Stress Level", f"Stress level for data point {random_index} from test sample: {stress_level}")
    messagebox.showinfo("Power", p_value)
    if p_value < 0.10:
        messagebox.showinfo("DOCTOR SUPERVISION", s)
    else:
        messagebox.showinfo("DOCTOR SUPERVISION", p)


def show_sample_data():
    data_type = "wt_filtered"
    test_type = "Arithmetic"

    dataset_ = load_dataset(data_type=data_type, test_type=test_type)
    dataset = split_data(dataset_, vi.SFREQ)

    random_index = np.random.randint(0, 96)
    random_data = dataset[random_index]

    plt.figure(figsize=(8, 6))
    plt.plot(random_data)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.title("Sample Data")
    plt.show()


def train_and_predict():
    x, x_test, y, y_test = train_test_split(features, label, test_size=0.2, random_state=1)
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=1)

    param_grid = {
        'C': [0.1, 1, 10, 100, 1000],
        'kernel': ['rbf']
    }

    split_index = [-1 if x in range(len(x_train)) else 0 for x in range(len(x))]
    psi = PredefinedSplit(test_fold=split_index)

    svm_clf = GridSearchCV(SVC(), param_grid, cv=psi, refit=True)
    svm_clf.fit(x, y)

    y_pred_test = svm_clf.predict(x_test)
    cm = confusion_matrix(y_test, y_pred_test)

    print("Confusion Matrix:")
    print(cm)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

    messagebox.showinfo("Results", f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_test)}\n\nAccuracy: {accuracy_score(y_test, y_pred_test)}")


# Create the main GUI window
window = tk.Tk()
window.title("STRESS LEVEL PREDICTION")
window.geometry("300x250")

# Add buttons to perform actions
calculate_button = tk.Button(window, text="Calculate Stress Level", command=calculate_stress_level)
calculate_button.pack(pady=10)



window.mainloop()

